#### This notebook presents how to perform maximum-likelihood parameter estimation for a single neuron.

In [145]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
import sys
sys.path.append("../")
sys.path.append("../code/")

In [147]:
import filters
import likelihood_functions as lk
import PoissonProcessClasses as PP

In [148]:
import imp
imp.reload(filters)
imp.reload(lk)

<module 'likelihood_functions' from '../likelihood_functions.py'>

In [149]:
import sys
sys.path.append("../")
sys.path.append("../data/")

Reading input-output data:

In [150]:
# reading stimulus
Stim = pd.read_csv('../data/Stim.csv',header = None)
# reading location of spikes
tsp = np.hstack(np.array(pd.read_csv('../data/tsp.csv',header = None)))

Extracting a spike train from spike positions:

In [151]:
dt = 0.01
tsp_int = np.ceil((tsp - dt*0.001)/dt)
tsp_int = np.reshape(tsp_int,(tsp_int.shape[0],1))
tsp_int = tsp_int.astype(int)
y = np.array([item in tsp_int for item in np.arange(Stim.shape[0]/dt)]).astype(int)

Creating filters:

In [152]:
# create a stimulus filter
kpeaks = np.array([0,round(20/3)])
pars_k = {'neye':5,'n':5,'kpeaks':kpeaks,'b':3}
K,K_orth,kt_domain = filters.createStimulusBasis(pars_k, nkt = 20) 

In [153]:
# create a post-spike filter
hpeaks = np.array([0.1,2])
pars_h = {'n':5,'hpeaks':hpeaks,'b':.4}
H,H_orth,ht_domain = filters.createPostSpikeBasis(pars_h,dt)

Conditional Intensity (spike rate):
$$\lambda_{\beta} = \exp(K(\beta_k)*Stim + H(\beta_h)*y)$$

Creating a matrix of covariates:

In [154]:
M = lk.construct_covariates(Stim,tsp,K,H_orth,dt)

Condtional intensity as a function of the covariates:
$$ \lambda_{\beta} = \exp(M\beta) $$

Create a Poisson process model with this intensity:

In [ ]:
model = PP.PPModel(M.T,dt = dt)

Setting initial parameters:

In [155]:
coeff_k0 = np.array([ 0.061453,0.284916,0.860335,1.256983,0.910615,0.488660,-0.887091,0.097441,0.026607,-0.090147])

coeff_h0 = np.zeros((5,))

pars0 = np.hstack((coeff_k0,coeff_h0))

Fitting the likelihood:

In [ ]:
res = model.fit(y,start_coef = pars0)

In [156]:
theta_MLE = res.x